## Troubleshooting (fixing any kind of problem - ususally problems in the system running the application) vs debugging (fixing the code)


- `tcpdump`, `Wireshark` = ongoing network connections, analyze the traffic

- `ps`, `top`, `free` = number and type of resourses used in a system

- `strace` - **system calls** made by a program
    - **system calls** - calls that the programs running on our computer make to the running kernel

- `ltrace` - look at the library calls made by software


### 0. get information
### 1. try to have reproduction case
### 2. finding the root cause of the problem
### 3. necessary remediation - shirt term (clean fan)
### 4. necessary remediation - long-term (monitoring system for overheating)

- documenting what we do is a good practice C:

In [1]:
# works only on linux
! strace -o outputfile.strace python test.py

/bin/bash: strace: command not found


In [ ]:
# to scroll through a lot of text output after tracing system calls of a script + 
# ctrl+G
! strace ./script.py | less  

### Good questions for reporting the issue (basically github Q):
- what were you trying to do?
- what steps did you follow?
- what was the expected result?
- what was the actual result?

### Elimination process:

- Consider simplest explanations first, unless it is directly needed (but not to assume right away that it's the problem that we solved before, still ask Q!)
- Try to reproduce the issue (rules out i.e. users computer as a problem - check whether the problem is related to i.e. specific website or not)

- Investigate further:
    - I.e. problem with server - running on linux machiene - connect using ssh
    - Run `top` command (shows state of the computer and processed the most CPU)

Load average:
how much time processer is busy in a given minute (1 - busy for whole minute, so normally should not be above the amount of the processes of the computer), a **number higher than the amount of processes means that computer is overloaded** -> take a look at the processes -> find that most of the CPU time is spent waiting (processes are stuck waiting for the OS to return from system calls) - usually happends when processes stuck getting the data from hard drive or the network -> look at the list of processes -> backu system is currently running on the server & seems to be using a lot of processing time -> kill -STOP BackupSystem (suspend the execution until decide to terminate/continue the process) -> check for load again -> processes are not stuck waiting for I/O -> success C: (immediate remidiation)


### Reproduction case:

- a way to verify if the problem is present or not
- sometimes can be tricky

### What if the issue does not reproduce (i.e. everything works fine) 
-> maybe the problem from user's side, ie. network routing|old config file intefering with a new version of the program|permissions problem|hardware: need to narrow it down

1. #### Read the logs avaliable
- Linux: sysyemlogs at /var/log/syslog/ and user-specific logs the .xsession-errors located at the home dir
- MacOS: on top of the system logs: /Library/Logs
- Windows: use EventViewer tool

2. #### If message is unhelpful/no error message, try to isolate the conditions that trigger the issue 
(other usr have same problem? same thing if usr logs into a different computer? config directory?)

3. #### Clear reproduction case -> next step - finding the root cause

### Find the root cause
cycle over:
1. look up the info we have
2. come up with hypothesis that could explain the problem
3. if does not explain the problem -> go to 1.

Whenever possible, check the hypothesis in a test environment, instead of the production env that our users are working with. -> avoid intefering with usr  and see, wether the issue gets reproduced in test env


## Tools to use in test env

### too much disk Input, output
- `iotop` (which processes are using input and output)
- `iostat` - statistics on input/output operations
- `vmstat` - statistics on virtual memory operations
- `ionice` - make process i.e. back up to reduce the priority to acess the disk and let the web services use it too (if produces too much input/output)

### server uses too much Network

- `iftop` - shows the current traffic on the network interfaces
-  `rsync -bwlimit` (if backup has the option to limit the bandwidth) or `Trickle` program

### compression alg for backup is too agressive (uses max of service power)

- `nice` - to reduce the priority of the process in accessing the CPU
or reduce the comprassion rate




### Reoccuring bugs that are hard to catch -> do extensive logging!

- logging configuration 
    - load on the computer
    - processes running at the same time
    - the usage of the network
- modify the code to get more information
- debugging mode (yeppp)

### Intermittent issues:

- **Heisenbugs** are hard to catch -> observer effect: just observing the error alters its state -> usually point ot bad resourse management (i.e. memorry wrongly allocated,network connections weren't correctly initialized, open files were not properly handled) -> take a look at the effected code
- **turn on/off** bugs :D - if **power cycling** releases resources stored in cache or memory, which gets rid of the problem - releasing all allocated memory, deleting tmp files, resetting the running state of problems, re-establishing network connections, closing open files... -> bug with software -> try to find fix or even automatic restart in the worst case
